In [1]:
if 'google.colab' in str(get_ipython()):
  # install packages required for this tutorial
  !pip install tensorflow==2.3.1
  !pip install tensorflow_quantum==0.4.0
  !pip install quple==0.7.6

# Tutorial-05 Encoding Circuit

In this tutorial, you will learn how to:

- Create a data encoding circuit with build-in templates
- Create customized data encoding circuit using template circuit blocks

# Data Encoding Circuit

A data encoding circuit is a quantum circuit for the encoding of classical data into the quantum state of the circuit qubits. It can be interpreted as a feature map $\mathbf{x} \rightarrow U_{\phi(\mathbf{x})}|0\rangle^{\otimes n}$ to the Hilbert space of $n$ qubits, where $\phi$ is an encoding function which transforms the data vector into the circuit parameters. Specifically, $U_{\phi (\mathbf{x})}$ is implemented as a series of unitary gate operations in the quantum circuit. There are several ways to encode data into qubits and each one provides different expressive power to the original data. 

The encoding circuits implemented by Quple consists of layers of unitary operators of the form $\exp(i\psi(x)\Sigma)H^{\times n}$ where $\psi$ is a data encoding function, $\Sigma$ is a generalized Pauli operator from the general Pauli group $G_n$ which is an $n$-fold tensor product of Pauli operators on $n$ qubits, and $x = (x_1, \dots , x_n )$ are the input features to be encoded. Layers of Pauli operators may be repeated several times (called the circuit depths) to the increase frequency spectrum of the final quantum state and thereby the expressivity of a quantum model equipped with the encoding circuit. 

The class that implements the encoding circuits are
* GeneralPauliEncoding
* GeneralPauliZEncoding
* FirstOrderPauliZEncoding
* SecondOrderPauliZEncoding

In [3]:
# import the quple encoding circuit modules
from quple.data_encoding import GeneralPauliEncoding
from quple.data_encoding import GeneralPauliZEncoding
from quple.data_encoding import FirstOrderPauliZEncoding
from quple.data_encoding import SecondOrderPauliZEncoding

### GeneralPauliEncoding

Arguments:
* feature_dimension (int): dimension of feature vector
* paulis (str, list of str): Pauli operations to be performed on each circuit block
* encoding_map (callable, default=None): data mapping function from $\mathbb{R}^{\text{feature dimension}} \rightarrow \mathbb{R}$. If None, the self product encoding function is used. 
* copies (int): number of times the circuit is repeated (circuit depth)

In [6]:
# Creates a general pruali encoding circuit with feature dimension 5 with circuit depth 2 using the Pauli 'Z'
encoding_circuit = GeneralPauliEncoding(feature_dimension=5, paulis=['Z'], copies=2)
encoding_circuit

(0, 0): ───H───Rz(pi*x_0)───H───Rz(pi*x_0)───

(0, 1): ───H───Rz(pi*x_1)───H───Rz(pi*x_1)───

(0, 2): ───H───Rz(pi*x_2)───H───Rz(pi*x_2)───

(0, 3): ───H───Rz(pi*x_3)───H───Rz(pi*x_3)───

(0, 4): ───H───Rz(pi*x_4)───H───Rz(pi*x_4)───

In [7]:
# Creates a general pruali encoding circuit with feature dimension 3 with circuit depth 1 using the Pauli 'X' and 'XX
encoding_circuit = GeneralPauliEncoding(feature_dimension=3, paulis=['X', 'XX'], copies=1)
encoding_circuit

(0, 0): ───H───Rx(pi*x_0)───H───@──────────────────────@───H───H───@──────────────────────@───H──────────────────────────────────────
                                │                      │           │                      │
(0, 1): ───H───Rx(pi*x_1)───H───X───Rz(pi*<x_0*x_1>)───X───H───────┼──────────────────────┼───H───────@──────────────────────@───H───
                                                                   │                      │           │                      │
(0, 2): ───H───Rx(pi*x_2)──────────────────────────────────────H───X───Rz(pi*<x_0*x_2>)───X───H───H───X───Rz(pi*<x_1*x_2>)───X───H───

### GeneralPauliZEncoding

A special case of `GeneralPauliEncoding` with Pauli operations composing of various orders of `Z` 

Arguments:
* feature_dimension (int): dimension of feature vector
* z_order (int): Order of pauli z operations to be performed on each circuit block
* encoding_map (callable, default=None): data mapping function from $\mathbb{R}^{\text{feature dimension}} \rightarrow \mathbb{R}$. If None, the self product encoding function is used. 
* copies (int): number of times the circuit is repeated (circuit depth)

In [8]:
# Creates a general pruali z encoding circuit with feature dimension 4 with circuit depth 2 using the Pauli 'Z'
encoding_circuit = GeneralPauliZEncoding(feature_dimension=4, z_order=1, copies=2)
encoding_circuit

(0, 0): ───H───Rz(pi*x_0)───H───Rz(pi*x_0)───

(0, 1): ───H───Rz(pi*x_1)───H───Rz(pi*x_1)───

(0, 2): ───H───Rz(pi*x_2)───H───Rz(pi*x_2)───

(0, 3): ───H───Rz(pi*x_3)───H───Rz(pi*x_3)───


In [9]:
# Creates a general pruali z encoding circuit with feature dimension 4 with circuit depth 1 using the Pauli 'Z', 'ZZ' and 'ZZZ'
encoding_circuit = GeneralPauliZEncoding(feature_dimension=3, z_order=3, copies=1)
encoding_circuit

(0, 0): ───H───Rz(pi*x_0)───@──────────────────────@───@──────────────────────@──────────────────────────────@──────────────────────────────────@───
                            │                      │   │                      │                              │                                  │
(0, 1): ───H───Rz(pi*x_1)───X───Rz(pi*<x_0*x_1>)───X───┼──────────────────────┼───@──────────────────────@───X───@──────────────────────────@───X───
                                                       │                      │   │                      │       │                          │
(0, 2): ───H───Rz(pi*x_2)──────────────────────────────X───Rz(pi*<x_0*x_2>)───X───X───Rz(pi*<x_1*x_2>)───X───────X───Rz(pi*<x_0*x_1*x_2>)───X───────

### FirstOrderPauliZEncoding

A special case of `GeneralPauliZEncoding` with `z_order=1`

Arguments:
* feature_dimension (int): dimension of feature vector
* encoding_map (callable, default=None): data mapping function from $\mathbb{R}^{\text{feature dimension}} \rightarrow \mathbb{R}$. If None, the self product encoding function is used. 
* copies (int): number of times the circuit is repeated (circuit depth)

In [10]:
# Creates a general pruali z encoding circuit with feature dimension 5 with circuit depth 2 using the Pauli 'Z'
encoding_circuit = FirstOrderPauliZEncoding(feature_dimension=5, copies=2)
print(encoding_circuit)

(0, 0): ───H───Rz(pi*x_0)───H───Rz(pi*x_0)───

(0, 1): ───H───Rz(pi*x_1)───H───Rz(pi*x_1)───

(0, 2): ───H───Rz(pi*x_2)───H───Rz(pi*x_2)───

(0, 3): ───H───Rz(pi*x_3)───H───Rz(pi*x_3)───

(0, 4): ───H───Rz(pi*x_4)───H───Rz(pi*x_4)───


### SecondOrderPauliZEncoding

A special case of `GeneralPauliZEncoding` with `z_order=2`

Arguments:
* feature_dimension (int): dimension of feature vector
* encoding_map (callable, default=None): data mapping function from $\mathbb{R}^{\text{feature dimension}} \rightarrow \mathbb{R}$. If None, the self product encoding function is used. 
* copies (int): number of times the circuit is repeated (circuit depth)

In [11]:
# Creates a general pruali z encoding circuit with feature dimension 3 with circuit depth 2 using the Pauli 'Z' and 'ZZ'
encoding_circuit = SecondOrderPauliZEncoding(feature_dimension=3, copies=2)
encoding_circuit

(0, 0): ───H───Rz(pi*x_0)───@──────────────────────@───@──────────────────────@──────────────────────────H───Rz(pi*x_0)────────────────@──────────────────────@───@──────────────────────@──────────────────────────────
                            │                      │   │                      │                                                        │                      │   │                      │
(0, 1): ───H───Rz(pi*x_1)───X───Rz(pi*<x_0*x_1>)───X───┼──────────────────────┼───@──────────────────────@───H────────────Rz(pi*x_1)───X───Rz(pi*<x_0*x_1>)───X───┼──────────────────────┼───@──────────────────────@───
                                                       │                      │   │                      │                                                        │                      │   │                      │
(0, 2): ───H───Rz(pi*x_2)──────────────────────────────X───Rz(pi*<x_0*x_2>)───X───X───Rz(pi*<x_1*x_2>)───X───H────────────Rz(pi*x_2)──────────────────────────────X───Rz(pi*<x_0*x_2>)───X───X───Rz(pi*<x_1*x_2>)───X───

### Use of a different encoding map

In [12]:
from quple.data_encoding.encoding_maps import distance_measure
# Creates a general pruali z encoding circuit with feature dimension 3 with circuit depth 1 using the Pauli 'Z' and 'ZZ' using the distance measure encoding map
encoding_circuit = SecondOrderPauliZEncoding(feature_dimension=3, copies=1, encoding_map=distance_measure)
print(encoding_circuit)

(0, 0): ───H───Rz(pi*x_0)───@────────────────────────────@───@────────────────────────────@────────────────────────────────────
                            │                            │   │                            │
(0, 1): ───H───Rz(pi*x_1)───X───Rz(pi*<x_0/2 - x_1/2>)───X───┼────────────────────────────┼───@────────────────────────────@───
                                                             │                            │   │                            │
(0, 2): ───H───Rz(pi*x_2)────────────────────────────────────X───Rz(pi*<x_0/2 - x_2/2>)───X───X───Rz(pi*<x_1/2 - x_2/2>)───X───


## Create custom encoding circuit

### 1. Using the base class `EncodingCircuit`

Usage is similar to `ParameterisedCircuit`

In [16]:
from quple.data_encoding import EncodingCircuit
encoding_circuit = EncodingCircuit(feature_dimension=4, copies=1, rotation_blocks=['RX', 'RY'], entanglement_blocks=['XX', 'YY'])
encoding_circuit

┌────────────┐                              ┌────────────┐
(0, 0): ───Rx(pi*<x_0/pi>)───Ry(pi*<x_0/pi>)───XX───────XX────────XX───────────────────────YY───────YY────────YY────────────────────────────────
                                               │        │         │                        │        │         │
(0, 1): ───Rx(pi*<x_1/pi>)───Ry(pi*<x_1/pi>)───XX^x_0───┼─────────┼─────XX────────XX───────YY^x_0───┼─────────┼─────YY────────YY────────────────
                                                        │         │     │         │                 │         │     │         │
(0, 2): ───Rx(pi*<x_2/pi>)───Ry(pi*<x_2/pi>)────────────XX^x_1────┼─────XX^x_3────┼────────XX───────YY^x_1────┼─────YY^x_3────┼────────YY───────
                                                                  │               │        │                  │               │        │
(0, 3): ───Rx(pi*<x_3/pi>)───Ry(pi*<x_3/pi>)──────────────────────XX^x_2──────────XX^x_4───XX^x_5─────────────YY^x_2──────────YY^x_4───YY^x_5───
                                                                 └────────────┘                              └────────────┘

2. Create a template circuit block

Using the `TemplateCircuitBlock` class, one can customize a circuit block to be applied in the entanglement layer

In [22]:
from typing import Sequence
import numpy as np

from quple import TemplateCircuitBlock

# create the RISWAP circuit block which consist of a parameterised RXX followed by a parameterised RYY block

class RISWAPBlock(TemplateCircuitBlock):
    
    @staticmethod
    def RYY(circuit:'quple.ParameterisedCircuit', theta, qubits:Sequence[int]):
        circuit.RX(np.pi/2, list(qubits))
        circuit.CX(tuple(qubits))
        circuit.RZ(theta, qubits[1])
        circuit.CX(tuple(qubits))
        circuit.RX(-np.pi/2, list(qubits))
        
    @staticmethod
    def RXX(circuit:'quple.ParameterisedCircuit', theta, qubits:Sequence[int]):
        circuit.H(list(qubits))
        circuit.CX(tuple(qubits))
        circuit.RZ(theta, qubits[1])
        circuit.CX(tuple(qubits))
        circuit.H(list(qubits))
        

    def build(self, circuit:'quple.ParameterisedCircuit', qubits:Sequence[int]):
        theta = circuit.new_param()
        RISWAPBlock.RXX(circuit, theta, qubits)
        RISWAPBlock.RYY(circuit, theta, qubits)
    
    @property
    def num_block_qubits(self) -> int:
        return 2

In [24]:
encoding_circuit = EncodingCircuit(feature_dimension=4, copies=1, entanglement_blocks=[RISWAPBlock()], entangle_strategy='linear')
encoding_circuit

(0, 0): ───H───H───@─────────────────────@───H───Rx(0.5π)───@─────────────────────@───Rx(-0.5π)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   │                     │                  │                     │
(0, 1): ───H───H───X───Rz(pi*<x_0/pi>)───X───H───Rx(0.5π)───X───Rz(pi*<x_0/pi>)───X───Rx(-0.5π)───H───@─────────────────────@───H───Rx(0.5π)───@─────────────────────@───Rx(-0.5π)──────────────────────────────────────────────────────────────────────────────────────
                                                                                                      │                     │                  │                     │
(0, 2): ───H──────────────────────────────────────────────────────────────────────────────────────H───X───Rz(pi*<x_1/pi>)───X───H───Rx(0.5π)───X───Rz(pi*<x_1/pi>)───X───Rx(-0.5π)───H───@─────────────────────@───H───Rx(0.5π)───@─────────────────────@───Rx(-0.5π)───
                                                                                                                                                                                         │                     │                  │                     │
(0, 3): ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────H───X───Rz(pi*<x_2/pi>)───X───H───Rx(0.5π)───X───Rz(pi*<x_2/pi>)───X───Rx(-0.5π)───